In [1]:
// declare our variables up here
let prompt;
let files;
let issue;


undefined

In [5]:
issue = `@file1:/path/to/file1
@file2:/path/to/file2

The amount of CPU in @file1 needs to be increased to 512M
The PVC storage amount in @file2 should be decreased to 10Gi
`;

'@file1:/path/to/file1\n' +
  '@file2:/path/to/file2\n' +
  '\n' +
  'The amount of CPU in @file1 needs to be increased to 512M\n' +
  'The PVC storage amount in @file2 should be decreased to 10Gi\n'

In [ ]:
files = new Map([
  ['file1', {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 10s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://my.new.openshift.datasource.svc.cluster.local:9091'\n"
  }],
  ['file2', {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: route.openshift.io/v1\n' +
      'metadata:\n' +
      '  name: grafana\n' +
      '  annotations:\n' +
      '    kubernetes.io/tls-acme: "true"\n' +
      'spec:\n' +
      '  host: grafana.operate-first.cloud\n' +
      '  to:\n' +
      '    kind: Service\n' +
      '    name: grafana-service\n' +
      '  port:\n' +
      '    targetPort: 3000\n',
    updatedContent: 'kind: Route\n' +
      'apiVersion: route.openshift.io/v1\n' +
      'metadata:\n' +
      '  name: grafana\n' +
      '  annotations:\n' +
      '    kubernetes.io/tls-acme: "true"\n' +
      'spec:\n' +
      '  host: grafana.operate-first.cloud\n' +
      '  to:\n' +
      '    kind: Service\n' +
      '    name: grafana-service\n' +
      '  port:\n' +
      '    targetPort: 3434\n' +
      '\n'
  }]
]);